In [1]:
import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchvision

In [1]:

from transformers import AutoTokenizer, AutoModel
from torchvision import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import csv
import pandas as pd



def preparedata(sentences, tokennumbers):
    
    RetVal=[]
    inputs=tokenizer(sentences, padding = True, truncation = True, return_tensors = "pt")
    outputs=aModel(**inputs)
    for i, tokens in enumerate(outputs[0]):
      RetVal.append(tokens[int(tokennumbers[i])])
      
    return RetVal

def casetonumber(case):
    switch = {
        "Nom": 0,
        "Acc": 1,
        "Ins": 2,
        "Ine": 3, 
        "Sup": 4,
        "Sub": 5
    }
    
    return switch.get(case, "Invalid case")


def gettokennumber(sentence, wordnumber):

    wordlist=sentence.split(" ")
    tokennumber = 0

    for i, word in enumerate(wordlist, 0):
        output=tokenizer(word, add_special_tokens=False)
        tokennumber = tokennumber + len(output["input_ids"])
        if i == int(wordnumber):
          break
    return tokennumber


In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
aModel = AutoModel.from_pretrained("bert-base-multilingual-cased", return_dict=True)

train_tsv=pd.read_csv('train.tsv',na_filter=None, quoting=3, sep="\t")
dev_tsv=pd.read_csv('dev.tsv',na_filter=None, quoting=3,sep="\t")



In [6]:
training_data=[]
test_data=[]

for i, obj in enumerate(train_tsv.values, 0):
      input=tokenizer(obj[0], return_tensors = "pt")
      output=aModel(**input)
      training_data.append([])
      training_data[i].append(output.last_hidden_state[0][gettokennumber(obj[0], obj[2])])
      training_data[i].append(casetonumber(obj[3]))


for i, obj in enumerate(dev_tsv.values):
      input=tokenizer(obj[0], padding = True, truncation = True, return_tensors = "pt")
      output=aModel(**input)

      test_data.append([])
      test_data[i].append(output.last_hidden_state[0][gettokennumber(obj[0], obj[2])])
      test_data[i].append(casetonumber(obj[3]))


trainloader=torch.utils.data.DataLoader(training_data, batch_size=16, shuffle=True)
testloader=torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True)
file1.close()
file2.close()

In [ ]:

class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, X):
        h = self.input_layer(X)
        h = self.relu(h)
        out = self.output_layer(h)
        return out

In [ ]:
Net = SimpleClassifier(
    input_dim = 768,
    output_dim = 6,
    hidden_dim = 50
)
Net
Net=Net.cuda()
NumEpoch=10

In [ ]:


def createLoss():
    return nn.CrossEntropyLoss()


def createOptimizer():
    return torch.optim.SGD(
        Net.parameters(), lr = 1e-1,
        momentum = 0.9, nesterov = True, 
        weight_decay = 1e-4)



def creatScheduler():
    return torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, numEpoch, eta_min=1e-2)



In [ ]:
def train(epoch):
    Net.train()
    running_loss = 0.0
    correct = 0.0
    total = 0

    for i, data in enumerate(trainloader, 0):
        tensors, labels = data
        
        tensors=tensors.cuda()
        labels=labels.cuda()

        optimizer.zero_grad()
        outputs=Net(tensors)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            running_loss +=loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            
            
    tr_loss = running_loss/i
    tr_corr = correct/total*100
    print("Train epoch " + str(epoch+1) + "  correct: " + str(tr_corr))
    return tr_loss, tr_corr

def val(epoch):
    Net.eval()
    running_loss=0.0
    correct = 0.0
    total=0

    for i, data in enumerate(testloader, 0):
        tensors, labels = data
        tensors=labels.cuda()
        inputs=inputs.cuda()
        
        with torch.no_grad():
            outputs=Net(inputs)
            loss = criterion(outputs, tensors)
            running_loss +=loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            
            
    val_loss = running_loss/i
    val_corr = correct/total * 100
    print("Test epoch " + str(epoch + 1) + " loss: " + str(running_loss / i) + " correct: " +  str(val_corr))
    return val_loss, val_corr

In [ ]:

train_accs = []
train_losses = []
val_accs = []
val_losses = []
best_acc = 0

torch.manual_seed(32)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark=False

criterion = createLoss()
optimizer = createOptimizer()
scheduler = creatScheduler()

for epoch in range(numEpoch):
    #Train
    loss, acc = train(epoch)
    train_accs.append(acc)
    train_losses.append(loss)

    #val
    loss, acc = val(epoch)
    val_accs.append(acc)
    val_losses.append(loss)
    scheduler.step()
    if acc>best_acc:
        best_acc = acc
        print("Best model so far")
        torch.save(Net, "model.pth")